# 0 - Instalation of modules

In [ ]:
#Run this cell to have the progress bar
!pip install tqdm
!pip install ipywidgets
!jupyter nbextension enable --py widgetsnbextension

# 1 - Importing modules

In [1]:
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm

# 2 - Get all files needed

In [4]:
data_folder = "../../Dataset/Movie/"

rating = pd.read_csv(f"{data_folder}ratings.csv").drop(["timestamp"], axis=1)
movie = pd.read_csv(f"{data_folder}movies.csv")

data = pd.merge(movie, rating, on="movieId")

# 3 - Transforming data

In [6]:
#Séparation en ligne du dataframe en petits dataframe
N = 1000
data_list = []
L = data.shape[0]

for i in range(1,N+1):
    debut = int((i-1)*L/N)
    fin = int(i*L/N)
    data_list.append(data.iloc[debut:fin, :])

In [11]:
#pivot table de chaque petits dataframe
pv_list = []
N = len(data_list)

i = 0
for d in tqdm(data_list):
    pv_list.append(d.pivot_table(index="userId", columns="title", values="rating", dropna=False))

  0%|          | 0/1000 [00:00<?, ?it/s]

In [8]:
#Réunir tous les pivots tables selon le film
titles = data.title.unique()
dico = {title : [] for title in titles}

for pv in pv_list:
    for title in pv:
        dico[title].append(pv[title])

In [24]:
#Concatener tous les pivots tables selon leur film et on les gardes uniquement si ils ont pas trop d'éléments manquant.
all_df = []
keep_title = []
thresh = 0.3

i = 0
for title in tqdm(titles):
    tmp = pd.concat(dico[title], axis=0)
    if not (np.sum(tmp.isna(), axis=0) > tmp.shape[0]*thresh):
        all_df.append(tmp)
        keep_title.append(title)


  0%|          | 0/10000 [00:00<?, ?it/s]

In [57]:
final_df = pd.DataFrame(index=np.sort(data.userId.unique()), columns=keep_title)

In [62]:
final_df

,"Matrix, The (1999)",Forrest Gump (1994),Pulp Fiction (1994),Star Wars: Episode IV - A New Hope (1977),Back to the Future (1985),Jurassic Park (1993),"Silence of the Lambs, The (1991)","Shawshank Redemption, The (1994)",Raiders of the Lost Ark (Indiana Jones and the Raiders of the Lost Ark) (1981),Star Wars: Episode VI - Return of the Jedi (1983),...,Harry Potter and the Half-Blood Prince (2009),"King's Speech, The (2010)",Gravity (2013),Star Wars: Episode VII - The Force Awakens (2015),How to Train Your Dragon (2010),"Piano, The (1993)",Sense and Sensibility (1995),Arrival (2016),Whiplash (2014),Intouchables (2011)
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
162537,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
162538,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
162539,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
162540,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [68]:
for df in tqdm(all_df):
    index = df.index
    values = df.values
    title = all_df[0].name

    for i in tqdm(range(len(index))):
        final_df.iloc[index[i]-1][title] = values[i]

  0%|          | 0/131 [00:00<?, ?it/s]

  0%|          | 0/94593 [00:00<?, ?it/s]

C:\Users\csis7\AppData\Local\Temp\ipykernel_14696\1443536587.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_df.iloc[index[i]-1][title] = values[i]


  0%|          | 0/97255 [00:00<?, ?it/s]

  0%|          | 0/98594 [00:00<?, ?it/s]

...

  0%|          | 0/21293 [00:00<?, ?it/s]

  0%|          | 0/23655 [00:00<?, ?it/s]

  0%|          | 0/12731 [00:00<?, ?it/s]

  0%|          | 0/15632 [00:00<?, ?it/s]

  0%|          | 0/17209 [00:00<?, ?it/s]

In [71]:
final_df.shape

(162541, 2420)

In [72]:
np.sum(final_df.isna(), axis=0)

Matrix, The (1999)                           162541
Forrest Gump (1994)                          162541
Pulp Fiction (1994)                          162541
Star Wars: Episode IV - A New Hope (1977)    162541
Back to the Future (1985)                    162541
                                              ...  
5050                                              0
5051                                              0
5052                                              0
5053                                              0
5055                                              0
Length: 2420, dtype: int64